In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from tensorflow import keras

In [2]:
data = pd.read_csv('FULLUwi.csv')

In [ ]:
data.head()

In [4]:
list_features = data.columns
print('They are',len(list_features),'features in the dataset.')
print('----------------')
for f in list_features:
    print('feature:', f, '|| Type:', type(data[f][0]), '|| Example:', data[f][0], '|| number of unique values', len(data[f].unique()) )

They are 14 features in the dataset.
----------------
feature: Subject || Type: <class 'str'> || Example: ACCT || number of unique values 194
feature: CourseCode || Type: <class 'numpy.int64'> || Example: 1005 || number of unique values 1256
feature: CourseType || Type: <class 'str'> || Example: E || number of unique values 17
feature: CourseType1 || Type: <class 'str'> || Example: E11 || number of unique values 477
feature: Semester || Type: <class 'numpy.int64'> || Example: 1 || number of unique values 4
feature: Year || Type: <class 'str'> || Example: 2021/2022 || number of unique values 5
feature: Faculty || Type: <class 'str'> || Example: FSS || number of unique values 36
feature: Level || Type: <class 'str'> || Example: 1 || number of unique values 11
feature: Location || Type: <class 'str'> || Example: Mona - Weekend || number of unique values 18
feature: Lecturer || Type: <class 'str'> || Example: Paul, Dwayney (Primary)  || number of unique values 4946
feature: Students || Typ

In [5]:
data.isnull().sum()

Subject            0
CourseCode         0
CourseType         2
CourseType1        0
Semester           0
Year               0
Faculty          718
Level              0
Location           0
Lecturer        4247
Students           0
Seats              0
Room             718
Attribute      50160
dtype: int64

In [6]:
def extraLec(name):
  if len(name.split())>3:
    return 'Yes'
  else:
    return 'No'

In [7]:
values = {'CourseType':'None','Faculty':'UN','Lecturer':'UN','Room':'UN','Attribute':'None'}
data= data.fillna(value=values)
data['Lecturer2'] = data.apply(lambda x: extraLec(x.Lecturer), axis = 1)
data['Lecturer']  = data.apply(lambda x: ' '.join(x.Lecturer.split(' (Primary)')[0].split()[-2:]), axis=1)

In [8]:
column = ['Attribute']
dataframe = data.loc[:,~data.columns.isin(column)].copy()
dataframe['Lecturer'] = dataframe.apply(lambda x: x.Lecturer=='Paul, Dwayney',axis =1)
labels = dataframe.pop("Lecturer")
feature_space = tf.keras.utils.FeatureSpace(
    features={
        "Subject": "string_categorical",
        "CourseCode": "integer_categorical",
        "CourseType": "string_categorical",
        "CourseType1": "string_categorical",
        "Semester": "integer_categorical",
        "Year": "string_categorical",
        "Level": "string_categorical",
        "Location": "string_categorical",
        "Room": "string_categorical",
        "Lecturer2": "string_categorical",
        "Faculty": "string_categorical",
        "Students": "float_normalized",
        "Seats": "float_normalized"
    },
    output_mode="concat",
)
dataset = tf.data.Dataset.from_tensor_slices(dict(dataframe))
# Before you start using the FeatureSpace,
# you must `adapt()` it on some data.
feature_space.adapt(dataset)

# You can call the FeatureSpace on a dict of data (batched or unbatched).
output_vector = feature_space(dict(dataframe))

In [9]:
X_train = output_vector[:40000]
X_test = output_vector[40000:]
y_train = labels[:40000]
y_test = labels[40000:]

In [10]:
tf.random.set_seed(42)
model = keras.models.Sequential([
 keras.layers.Dense(150, activation="relu",  input_shape=X_train.shape[1:]),
 keras.layers.Dense(1, activation="sigmoid")
])
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
 restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_model.h5", save_best_only=True)
opt = keras.optimizers.Nadam(learning_rate=0.01)
model.compile(loss="binary_crossentropy", optimizer = 'sgd', metrics=['accuracy']) #limited + general
history = model.fit(X_train, y_train, epochs=300,
 validation_data=(X_test, y_test), callbacks=[early_stopping_cb,model_checkpoint_cb])

Epoch 1/300
1250/1250 [==============================] - 8s 6ms/step - loss: 0.0573 - accuracy: 0.9984 - val_loss: 0.0093 - val_accuracy: 0.9997
Epoch 2/300
1250/1250 [==============================] - 6s 5ms/step - loss: 0.0061 - accuracy: 0.9996 - val_loss: 0.0052 - val_accuracy: 0.9997
Epoch 3/300
1250/1250 [==============================] - 9s 7ms/step - loss: 0.0045 - accuracy: 0.9996 - val_loss: 0.0040 - val_accuracy: 0.9997
Epoch 4/300
1250/1250 [==============================] - 8s 6ms/step - loss: 0.0039 - accuracy: 0.9996 - val_loss: 0.0034 - val_accuracy: 0.9997
Epoch 5/300
1250/1250 [==============================] - 7s 5ms/step - loss: 0.0036 - accuracy: 0.9996 - val_loss: 0.0031 - val_accuracy: 0.9997
Epoch 6/300
1250/1250 [==============================] - 8s 6ms/step - loss: 0.0034 - accuracy: 0.9996 - val_loss: 0.0028 - val_accuracy: 0.9997
Epoch 7/300
1250/1250 [==============================] - 6s 5ms/step - loss: 0.0033 - accuracy: 0.9996 - val_loss: 0.0027 - val_ac

In [11]:
predictions = model.predict(output_vector)
predictions = (predictions > 0.5).astype(int)

1884/1884 [==============================] - 4s 2ms/step


In [12]:
data['PreLec'] = predictions
data.to_csv('LTest.csv',index=False)